In [ ]:
from Bio.Alphabet.IUPAC import ExtendedIUPACProtein
from collections import Counter
import numpy as np

def counts1d(ar, values):
    return np.bincount(ar, minlength=max(values) + 1)[values]

def char_counts(ar, axis=None, chars=None, return_chars=False):
    ar = np.asanyarray(ar, 'c')
    out_dtype = ar.dtype
    if not chars:
        chars = np.unique(ar)
    else:
        chars = np.asanyarray(chars, ar.dtype)
    numeric_ar = ar.view('uint8')
    numeric_chars = chars.view('uint8')
    count_slice = lambda slice: counts1d(slice, numeric_chars)
    out = np.apply_along_axis(count_slice, axis, numeric_ar)
    if return_chars:
        return out.T, numeric_chars.view(out_dtype)
    else:
        return out.T
    
def aln_profile(alignment, alphabet=None, return_chars=False):
    return char_counts(alignment, chars=alphabet,
                       axis=0, return_chars=return_chars)

def aln_entropy(alignment):
    prof = aln_profile(alignment)
    pk = prof / np.sum(prof, axis=1).reshape(-1, 1)
    
    # log of the frequency of each character, but if the character
    # is not counted, the value is 0, not -Inf or NaN
    logpk = np.zeros_like(pk)
    logpk[pk.nonzero()] = np.log(pk[pk.nonzero()])
    
    entropy = -np.sum(pk * logpk, axis=1)
    return entropy

def print_aln(alignment):
    for rec in alignment:
        print(rec.seq)

In [ ]:
from Bio.AlignIO import read as read_aln
from Bio.Alphabet import generic_protein, generic_dna
aln = read_aln('seq/refs.hmmalign.refine.afa', 'fasta', alphabet=generic_protein)

In [ ]:
from matplotlib import pyplot as plt

plt.plot(aln_entropy(aln))

In [ ]:
def moving_window(x, size, f):
    x = np.asarray(x)
    out = np.empty(len(x))
    for i in range(len(x)):
        start = i
        stop  = min(len(x), i + size)
        out[i] = f(x[start:stop])
    return out

def smooth(x, bw):
    return moving_window(x, bw, f = np.mean)

In [ ]:
ax = plt.subplot(111)
ax.plot(smooth(aln_entropy(aln), 5))
ax.axvspan(350, 375, color='k', alpha=0.1)
ax.axvspan(515, 535, color='k', alpha=0.1)

In [ ]:
trace = aln_entropy(aln)

ax = plt.subplot(111)
ax.plot(trace)

regions = [(360, 369),  # Perfect AA conservation
           (442, 448),  # Perfect, but short (maybe 1 more 5' base)
           (493, 500),  # Imperfect and short
           (520, 531),  # Perfect AA conservation
          ]

min_max = lambda lst: (min(lst), max(lst))
minimum, maximum = min_max([coord for reg in regions
                                  for coord in reg])
ax.set_xlim(minimum - 10, maximum + 10)

for start, stop in regions:
    ax.axvspan(start, stop, color='g', alpha=0.15)

In [ ]:
consensus(prot[:,440:450])

In [ ]:
from collections import Counter

def consensus(alignment):
    prof, chars = aln_profile(alignment, return_chars=True)
    return chars[np.argmax(prof, axis=1)].astype(str)
    

for start, stop in regions:
    print("".join(consensus(aln[:,start:stop])))

In [ ]:
reverse_translation_table = \
{'F': 'TTY',
 'L': 'YTN',
 'I': 'ATH',
 'M': 'ATG',
 'V': 'GTN',
 'S': 'WSN',
 'P': 'CCN',
 'T': 'ACN',
 'A': 'GCN',
 'Y': 'TAY',
 'H': 'CAY',
 'Q': 'CAR',
 'N': 'AAY',
 'K': 'AAR',
 'D': 'GAY',
 'E': 'GAR',
 'C': 'TGY',
 'W': 'TGG',
 'R': 'MGN',
 'G': 'GGN',
}

def back_translate(prot_seq):
    out = []
    for res in prot_seq:
        out.append(reverse_translation_table[res])
    return "".join(out)

back_translate("FLIMVSPTAYHQNKDECWRG")
# TTYYTNATHATGGTNWSNCCNACNGCNTAYCAYCARAAYAARGAYGARTGYTGGMGNGGN

In [ ]:
from Bio.Seq import Seq
from Bio.Alphabet import IUPAC

for motif in (consensus(aln[:,start:stop]) for start, stop in regions):
    seq = Seq(back_translate(motif), alphabet=IUPAC.ambiguous_dna)
    print("{!s: <40}{!s: <40}".format(seq, seq.reverse_complement()))

In [ ]:
from Bio.AlignIO import read as read_aln
from Bio.Alphabet import generic_protein, generic_dna, Gapped
from Bio.Alphabet.IUPAC import protein, unambiguous_dna
import numpy as np

prot = read_aln('seq/refs.mcra-hmmaln.auto-refine.afa', 'fasta',
                alphabet=Gapped(protein))
nucl = read_aln('seq/refs.mcra-hmmaln.auto-refine.afn', 'fasta',
                alphabet=Gapped(unambiguous_dna))

In [ ]:
plt.plot(smooth(aln_entropy(nucl), 20))

In [ ]:
def nucl_freq_plot(xs, *args, ax=None, colors=None, **kwargs):
    total = np.sum(xs, axis=0)
    tally = np.zeros_like(total)
    pos = range(len(total))
    if not ax:
        ax = plt
    if not colors:
        colors = 'g', 'b', 'k', 'r', 'w'
    for values, c in zip(xs, colors):
        ax.bar(pos, values / total, bottom=tally / total, color=c)
        tally += values

In [ ]:
fig = plt.figure(figsize=(13,6))

for i, region in enumerate(regions, start=1):
    ax = fig.add_subplot(2, 2, i)
    nucl_coords = (region[0] * 3, (region[1] - 1) * 3)
    nucl_freq_plot(aln_profile(nucl[:,nucl_coords[0]:nucl_coords[1]]).T, ax=ax)
    ax.set_xlim(-5, 35)
    ax.set_title(nucl_coords)

In [ ]:
reverse_translation_table = {
    'F': 'TTY',
    'L': 'YTN',
    'I': 'ATH',
    'M': 'ATG',
    'V': 'GTN',
    'S': 'WSN',
    'P': 'CCN',
    'T': 'ACN',
    'A': 'GCN',
    'Y': 'TAY',
    'H': 'CAY',
    'Q': 'CAR',
    'N': 'AAY',
    'K': 'AAR',
    'D': 'GAY',
    'E': 'GAR',
    'C': 'TGY',
    'W': 'TGG',
    'R': 'MGN',
    'G': 'GGN',
}

codon_degen_table = {
    'F': 2,
    'L': 8,
    'I': 3,
    'M': 1,
    'V': 4,
    'S': 16,
    'P': 4,
    'T': 4,
    'A': 4,
    'Y': 2,
    'H': 2,
    'Q': 2,
    'N': 2,
    'K': 2,
    'D': 2,
    'E': 2,
    'C': 2,
    'W': 1,
    'R': 8,
    'G': 4,
}

codon_degen_table_3_trunc = {
    'F': 1,
    'L': 2,
    'I': 1,
    'M': 1,
    'V': 1,
    'S': 4,
    'P': 1,
    'T': 1,
    'A': 1,
    'Y': 1,
    'H': 1,
    'Q': 1,
    'N': 1,
    'K': 1,
    'D': 1,
    'E': 1,
    'C': 1,
    'W': 1,
    'R': 2,
    'G': 1,
}

codon_degen_table_5_trunc = {
    'F': 2,
    'L': 4,
    'I': 3,
    'M': 1,
    'V': 4,
    'S': 8,
    'P': 4,
    'T': 4,
    'A': 4,
    'Y': 2,
    'H': 2,
    'Q': 2,
    'N': 2,
    'K': 2,
    'D': 2,
    'E': 2,
    'C': 2,
    'W': 1,
    'R': 4,
    'G': 4,
}


In [ ]:
aln_profile(prot[:,regions[2][0]:regions[2][1]], return_chars=True)

In [ ]:
motifs = []

for start, stop in regions:
    motifs.append("".join(consensus(aln[:,start:stop])))
    
for motif in motifs:
    for aa in motif:
        print("{!s: <3}".format(aa), sep="", end="")
    print("\n", sep="", end="")
    for aa in motif:
        print("{!s: <3}".format(codon_degen_table[aa]), sep="", end="")
    print("\n", sep="", end="")
    print("\n", sep="", end="")

In [ ]:
def calc_degen(motif, reverse=False):
    tally = 1
    for i, aa in enumerate(motif):
        if reverse and (i == 0):
            tally *= codon_degen_table_5_trunc[aa]
        elif (not reverse) and (i == len(motif) - 1):
            tally *= codon_degen_table_3_trunc[aa]
        else:
            tally *= codon_degen_table[aa]
    return tally
    

def minimum_subseq_degen(motif, length, reverse=False):
    hypotheses = []
    for start in range(len(motif) - length):
        hypothesis = motif[start:start+length]
        hypotheses.append((hypothesis, calc_degen(hypothesis, reverse)))
    return hypotheses

minimum_subseq_degen('', 4, reverse=False)

In [ ]:
minimum_subseq_degen('DLQDQCG', 4, reverse=False)

In [ ]:
seq = Seq(back_translate('LQDQ'), alphabet=IUPAC.ambiguous_dna)
print("{!s: <40}{!s: <40}".format(seq, seq.reverse_complement()))